<a href="https://colab.research.google.com/github/federicoding/Airline_Satisfaction/blob/main/GoingOnlineTentativeV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

On va essayer de refaire le code de l'App de manière plus propre, et avec un @st.cache() pour le run.
On se tentera même un pickle!

#Apparté pour pickle le modèle

Je fais ça pour travailler avec des noms de variable corrects, sans espaces.

In [1]:
import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestClassifier

df = pd.read_csv(filepath_or_buffer='https://raw.githubusercontent.com/federicoding/Airline_Satisfaction/main/Airline_Dataset.csv',sep=';')

df['Customer Type'] = df['Customer Type'].map({'Loyal Customer':'Returning Customer', 'disloyal Customer':'First-time Customer'})

df = df.dropna(axis=0)

df['Departure Delay in Minutes'] = df['Departure Delay in Minutes'].astype('float')

df = df.rename(columns={'Leg room service':'Leg room'})

from string import capwords
df.columns = [capwords(i) for i in df.columns]
df = df.rename(columns={'Departure/arrival Time Convenient':'Departure/Arrival Time Convenience'})

df = df[(df['Inflight Wifi Service']!=0)&(df['Departure/Arrival Time Convenience']!=0)&(df['Ease Of Online Booking']!=0)&(df['Gate Location'])&(df['Food And Drink']!=0)&(df['Online Boarding']!=0)&(df['Seat Comfort']!=0)&(df['Inflight Entertainment']!=0)&(df['On-board Service']!=0)&(df['Leg Room']!=0)&(df['Baggage Handling']!=0)&(df['Checkin Service']!=0)&(df['Inflight Service']!=0)&(df['Cleanliness']!=0)]

df['Satisfaction'] = df['Satisfaction'].map({'satisfied':1,'neutral or dissatisfied':0})
df = df.reset_index()
df = df.drop('index',axis=1)
df['Total Delay'] = df['Departure Delay In Minutes'] + df['Arrival Delay In Minutes']

DF = df.copy()
df = df.drop('Id',axis=1)

df = df.reindex(columns=['Satisfaction']+list(df.columns)[:-2]+['Total Delay'])
df = df.drop(['Departure Delay In Minutes','Arrival Delay In Minutes'],axis=1)

df['Class'] = df['Class'].map({'Eco':'Economy','Eco Plus':'Economy','Business':'Business'})

df1 = pd.get_dummies(df,columns=['Gender','Customer Type','Type Of Travel','Class'],drop_first=True)

df1 = df1.drop(['Total Delay','Flight Distance','Age','Gate Location','Departure/Arrival Time Convenience',
                'Gender_Male'],axis=1)

df1.rename(columns= {'Inflight Wifi Service': 'Inflight_Wifi_Service',
                     'Ease Of Online Booking': 'Ease_Of_Online_Booking',
                     'Food And Drink': 'Food_And_Drink',
                     'Online Boarding': 'Online_Boarding',
                     'Seat Comfort': 'Seat_Comfort',
                     'Inflight Entertainment': 'Inflight_Entertainment',
                     'On-board Service': 'On_board_Service',
                     'Leg Room': 'Leg_Room',
                     'Baggage Handling': 'Baggage_Handling',
                     'Checkin Service': 'Checkin_Service',
                     'Inflight Service': 'Inflight_Service',
                     'Cleanliness': 'Cleanliness',
                     'Customer Type_Returning Customer': 'Customer_Type_Returning_Customer',
                     'Type Of Travel_Personal Travel': 'Type_Of_Travel_Personal_Travel',
                     'Class_Economy': 'Class_Economy'})

y = df1['Satisfaction']
X = df1.drop('Satisfaction',axis=1)

rf = RandomForestClassifier(max_depth=17, random_state=42)
rf.fit(X,y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=17, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [2]:
import pickle
pickle_out = open('Airline_satif_model.pkl', mode ='wb')
pickle.dump(rf, pickle_out)
pickle_out.close()

#Vrai code

In [3]:
%%writefile app5.py

import pickle
import streamlit as st

#loading the trained model
picle_in = open('Airline_satif_model.pkl', 'rb')
classifier = pickle.load(pickle_in)

@st.cache()
def prediction_func(Inflight_Wifi_Service,Ease_Of_Online_Booking,
               Food_And_Drink,Online_Boarding,Seat_Comfort,
               Inflight_Entertainment,On_board_Service,Leg_Room,
               Baggage_Handling,Checkin_Service,Inflight_Service,
               Cleanliness,Customer_Type_Returning_Customer,
               Type_Of_Travel_Personal_Travel,Class_Economy):
  #Make predictions
  prediction = classifier.predict(df)
  prediction_proba = classifier.predict_proba(df)

  return prediction, prediction_proba

# On tente un truc un peu joli

def main():
  #front end elements
  html_temp = """
  <div style ="background-color:yellow;padding:13px"> 
  <h1 style ="color:black;text-align:center;">Streamlit Airline Satisfaction Prediction ML App</h1> 
  </div> 
  """

  #display the front end aspect
  st.markdown(thml_temp, unsafe_allow_thml = True)

  #rajout des lignes pour selectionner le valeurs du voyageur
  def user_input_features():
    Inflight_Wifi_Service = st.selectbox('Inflight Wifi Service',[1,2,3,4,5])
    Ease_Of_Online_Booking = st.selectbox('Ease Of Online Booking',[1,2,3,4,5])
    Food_And_Drink = st.selectbox('Food And Drink',[1,2,3,4,5])
    Online_Boarding = st.selectbox('Online Boarding',[1,2,3,4,5])
    Seat_Comfort = st.selectbox('Seat Comfort',[1,2,3,4,5])
    Inflight_Entertainment = st.selectbox('Inflight Entertainment',[1,2,3,4,5])
    On_board_Service = st.selectbox('On-board Service',[1,2,3,4,5])
    Leg_Room = st.selectbox('Leg Room',[1,2,3,4,5])
    Baggage_Handling = st.selectbox('Baggage Handling',[1,2,3,4,5])
    Checkin_Service = st.selectbox('Checkin Service',[1,2,3,4,5])
    Inflight_Service = st.selectbox('Inflight Service',[1,2,3,4,5])
    Cleanliness = st.selectbox('Cleanliness',[1,2,3,4,5])
    Customer_Type_Returning_Customer = st.selectbox('Customer Type_Returning Customer',[0,1])
    Type_Of_Travel_Personal_Travel = st.selectbox('Type Of Travel_Personal Travel',[0,1])
    Class_Economy = st.selectbox('Class_Economy',[0,1])
    data = {'Inflight Wifi Service': Inflight_Wifi_Service,
            'Ease Of Online Booking': Ease_Of_Online_Booking,
            'Food And Drink': Food_And_Drink,
            'Online Boarding': Online_Boarding,
            'Seat Comfort': Seat_Comfort,
            'Inflight Entertainment': Inflight_Entertainment,
            'On-board Service': On_board_Service,
            'Leg Room': Leg_Room,
            'Baggage Handling': Baggage_Handling,
            'Checkin Service': Checkin_Service,
            'Inflight Service': Inflight_Service,
            'Cleanliness': Cleanliness,
            'Customer Type_Returning Customer': Customer_Type_Returning_Customer,
            'Type Of Travel_Personal Travel': Type_Of_Travel_Personal_Travel,
            'Class_Economy': Class_Economy}
    features = pd.DataFrame(data, index=[0])
    return features

  df = uset_input_features()

  #Bouton pour lancer la prédiction et store les values...

  if st.button('Predict'):
    result = prediction_func(df)
    st.success('Your Passenger prediction is {0}, with probability {1}'.format(result[0],result[1]))

if __name__=='__main__':
  main()


Writing app5.py
